In [6]:

%glue_version 4.0

# --- nn
# %region eu-central-1
# %iam_role arn:aws:iam::988864172292:role/custom_role_for_glue_interactive_sessions
# --- tgedr
%region eu-north-1
%iam_role arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service




Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.3 and you have 1.0.2 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Setting Glue version to: 4.0
Previous region: None
Setting new region to: eu-north-1
Region is set to: eu-north-1
Current iam_role is None
iam_role has been set to arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service.


In [9]:
%%configure
{
  "enable_spark_ui": "true",
  "python_version": "3.10"
}

The following configurations have been updated: {'enable_spark_ui': 'true', 'python_version': '3.10'}


In [1]:
spark

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 5
Session ID: 66db3e06-f352-4caf-8171-ee2dc3f7c587
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
--enable_spark_ui true
--python_version 3.10
Waiting for session 66db3e06-f352-4caf-8171-ee2dc3f7c587 to get into ready status...
Session 66db3e06-f352-4caf-8171-ee2dc3f7c587 has been created.


In [3]:
from pyspark.sql import functions as F

In [2]:
# parquet_file = "s3://988864172292-gscdm-dev-eu-central-1-landing-zone/dh_cdm/events"
parquet_file = "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering"

In [8]:
df = spark.read.parquet(parquet_file).limit(12)

In [12]:
df.collect()

[Row(symbol='AMD', variable='Adj Close', value=3.1458330154418945, actual_time=322099200.0), Row(symbol='AMD', variable='Close', value=3.1458330154418945, actual_time=322099200.0), Row(symbol='AMD', variable='High', value=3.3020830154418945, actual_time=322099200.0), Row(symbol='AMD', variable='Low', value=3.125, actual_time=322099200.0), Row(symbol='AMD', variable='Open', value=0.0, actual_time=322099200.0), Row(symbol='AMD', variable='Volume', value=219600.0, actual_time=322099200.0), Row(symbol='AMD', variable='Adj Close', value=3.03125, actual_time=322185600.0), Row(symbol='AMD', variable='Close', value=3.03125, actual_time=322185600.0), Row(symbol='AMD', variable='High', value=3.125, actual_time=322185600.0), Row(symbol='AMD', variable='Low', value=2.9375, actual_time=322185600.0), Row(symbol='AMD', variable='Open', value=0.0, actual_time=322185600.0), Row(symbol='AMD', variable='Volume', value=727200.0, actual_time=322185600.0)]


In [14]:
result = df.filter(F.col("variable").isin(["Adj Close", "Volume"])).groupBy(["symbol", "actual_time"]
       ).pivot("variable").max("value"
       ).withColumnRenamed("Adj Close", "adj_close"
       ).withColumnRenamed("Volume", "volume").sort("actual_time", ascending=True)

In [11]:
result.show()

+------+-----------+------------------+--------+
|symbol|actual_time|         Adj Close|  Volume|
+------+-----------+------------------+--------+
|   AMD| 3.220992E8|3.1458330154418945|219600.0|
|   AMD| 3.221856E8|           3.03125|727200.0|
+------+-----------+------------------+--------+


In [15]:
result.collect()

[Row(symbol='AMD', actual_time=322099200.0, adj_close=3.1458330154418945, volume=219600.0), Row(symbol='AMD', actual_time=322185600.0, adj_close=3.03125, volume=727200.0)]


In [18]:
df1 = (
       df[df["variable"].isin(["Adj Close", "Volume"])]
       ).groupBy(["symbol", "actual_time"]
       ).pivot("variable").max("value"
       ).sort("actual_time", ascending=True)

df1.show()

+------+-----------+------------------+---------+
|symbol|actual_time|         Adj Close|   Volume|
+------+-----------+------------------+---------+
|   AMD| 3.220992E8|3.1458330154418945| 219600.0|
|   AMD| 3.221856E8|           3.03125| 727200.0|
|   AMD|  3.22272E8|3.0416669845581055| 295200.0|
|   AMD| 3.223584E8|3.0104169845581055| 159600.0|
|   AMD| 3.224448E8|2.9166669845581055| 130800.0|
|   AMD|  3.22704E8|2.6666669845581055| 436800.0|
|   AMD| 3.227904E8|2.6041669845581055| 645600.0|
|   AMD| 3.228768E8|2.4479169845581055| 466800.0|
|   AMD| 3.229632E8|             2.375|1129200.0|
|   AMD| 3.230496E8|2.5416669845581055| 666000.0|
|   AMD| 3.233088E8|2.5416669845581055| 430800.0|
|   AMD| 3.233952E8|2.7395830154418945| 492000.0|
|   AMD| 3.234816E8|             2.625| 283200.0|
|   AMD|  3.23568E8|2.6354169845581055| 226800.0|
|   AMD| 3.239136E8|2.5416669845581055| 355200.0|
|   AMD|     3.24E8|           2.59375| 765600.0|
|   AMD| 3.240864E8|2.6666669845581055| 134400.0|


In [15]:
from pyspark.sql import functions as F
df1.groupBy(["symbol", "actual_time"]).pivot("variable").max("value").show()

+------+-----------+-------------------+----------+
|symbol|actual_time|          Adj Close|    Volume|
+------+-----------+-------------------+----------+
|   AMD| 3.547584E8| 4.2291669845581055|  413400.0|
|   AMD| 3.754944E8| 2.8958330154418945|  113400.0|
|   AMD| 3.929472E8| 3.7916669845581055|  178200.0|
|   AMD| 3.941568E8| 4.0416669845581055|  240600.0|
|   AMD| 4.342464E8|            16.0625|  620400.0|
|   AMD| 4.379616E8|              16.75|  512000.0|
|   AMD| 4.491936E8|             14.125|  469400.0|
|   AMD| 4.533408E8|             16.625|  441400.0|
|   AMD| 4.603392E8|            17.5625| 4924000.0|
|   AMD| 4.612896E8|            18.4375|  859000.0|
|   AMD| 4.796928E8|              15.25|  734200.0|
|   AMD| 5.152032E8|            14.1875|  701200.0|
|   AMD| 5.323104E8|             7.8125|  393400.0|
|  MSFT| 5.331744E8|0.10870875418186188|1.738368E8|
|   AMD| 5.332608E8|              7.875|  285400.0|
|  MSFT|  5.35248E8|0.10198217630386353|  1.2672E7|
|  MSFT| 5.3